In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
model = keras.models.load_model("xception_v4_large_08_0.894.h5")
tf.saved_model.save(model, "clothing-model")

INFO:tensorflow:Assets written to: clothing-model/assets


In [5]:
!saved_model_cli show --dir clothing-model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          input_8: Tensor

We are only interested in the following parts

- the signature definition, which is `serving_default`
- the input name, which is `input_8`
- the output name, which is `dense_7`

```
signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
```

In [8]:
# TF-serving usings gRPC for communication.
!pip install grpcio==1.32.0 tensorflow-serving-api==2.3.0 keras_image_helper==0.0.1

In [9]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2, prediction_service_pb2_grpc

ModuleNotFoundError: No module named 'grpc'

In [ ]:
host = "127.0.0.1:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [11]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor("xception", target_size=(299, 299))

url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)

In [ ]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(dat, shape=data.shape)

In [ ]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = "clothing-model"
pb_request.model_spec.signature_name = "serving_default"
pb_request.inputs["input_8"].CopyFrom(np_to_protobuff(X))

In [25]:
pb_result = stub.Predict(pb_request, timeout=20.0)
pred = pb_result.outputs["dense_7"].float_val

labels = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

result = {c: p for c, p in zip(labels, pred)}

NameError: name 'stub' is not defined

In [22]:
import requests

data = {
    "instances": X.tolist(),
}
res = requests.post("http://127.0.0.1:8501/v1/models/clothing-model:predict", json=data)
preds = res.json()
preds

{'predictions': [[-1.86829078,
   -4.76124477,
   -2.31698346,
   -1.06257033,
   9.88716125,
   -2.81243396,
   -3.66628313,
   3.20036173,
   -2.60233831,
   -4.83504677]]}

In [24]:
labels = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

result = {c: p for c, p in zip(labels, preds['predictions'][0])}
result

{'dress': -1.86829078,
 'hat': -4.76124477,
 'longsleeve': -2.31698346,
 'outwear': -1.06257033,
 'pants': 9.88716125,
 'shirt': -2.81243396,
 'shoes': -3.66628313,
 'shorts': 3.20036173,
 'skirt': -2.60233831,
 't-shirt': -4.83504677}